<a href="https://colab.research.google.com/github/Aljumaily/COSC5P30/blob/main/cosc5p70_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install graphiq

In [19]:

from v0 import CircuitOptimizer
#import CircuitOptimizer
import time
from networkx import Graph
from graphiq.circuit.ops import Hadamard, CNOT, Phase

engine: CircuitOptimizer = CircuitOptimizer(e=2, p=3, c=0)

engine.add(CNOT(control=0, control_type="p", target=1, target_type="p"))
engine.add(CNOT(control=0, control_type="p", target=1, target_type="p"))
engine.add(CNOT(control=0, control_type="p", target=2, target_type="p"))

#hadamards
engine.add(Hadamard(register=0, reg_type="p"))
engine.add(Hadamard(register=1, reg_type="p"))
engine.add(Hadamard(register=2, reg_type="p"))
# engine.add(SigmaZ(register=2, reg_type="p"))

# z gate error

#hadamards
engine.add(Hadamard(register=0, reg_type="p"))
engine.add(Hadamard(register=1, reg_type="p"))
engine.add(Hadamard(register=2, reg_type="p"))

#syndrome extraction
# engine.add(ClassicalCNOT(control=1, control_type="p", target=0, target_type="p", c_register=0))
engine.add(CNOT(control=0, control_type="p", target=0, target_type="e"))
engine.add(CNOT(control=1, control_type="p", target=0, target_type="e"))
engine.add(CNOT(control=1, control_type="p", target=1, target_type="e"))
engine.add(CNOT(control=2, control_type="p", target=1, target_type="e"))

# # engine.add(CNOT(control=0, control_type="p", target=3, target_type="p"))
# engine.add(
#     MeasurementCNOTandReset(
#         control=0, control_type="e", target=1, target_type="p", c_register=0
#     )
# )

engine.add(CNOT(control=0, control_type="p", target=2, target_type="p"))
engine.add(CNOT(control=0, control_type="p", target=1, target_type="p"))


engine.draw_circuit()

engine.get_graph_edges()
engine.plot_converted_numeric_graph()
start = time.time()
# engine.relabel_graph(engine.get_graph())
# engine.best_circuit_depth_of_isomorphic_graphs(100)
eq_lc_graphs: list[Graph] = engine.get_lc_equivalent_graphs(2)
engine.get_lowest_circuit_depth(eq_lc_graphs)
end = time.time()
length = end - start
print("It took", length, "seconds!")
#100+300+3000+30000+10000+1000




RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 640x480 with 1 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 640x480 with 1 Axes>

current graph score: 34


KeyboardInterrupt: 